In [ ]:
using RigidBodySim
using RigidBodyDynamics
using Plots
using LinearAlgebra
using Statistics
using SpecialFunctions
using Swarm
using Trajectory
using PerformanceIndex
include("src/fracionario.jl")
include("src/simulation.jl");

In [ ]:
urdf = "src/pelican.urdf"
robot = parse_urdf(urdf) # load dynamic
initial_state = MechanismState(robot) # inicial conditions 
num_joints = num_positions(initial_state); # joints quantity

In [ ]:
time_end = 2. #simulation time
Δt = 0.05  # discretization time
lb = vcat(fill(10., num_joints * 2), zeros(2) )# lower bound
ub = fill( 10000. ,num_joints * 2) #upper bound
ub = vcat(ub, ones(num_joints))
saturation_value = [200., 15.];

In [ ]:
# Trajectory generation

final_position = 0.8
xr, vr, ar, jr = minimumjerk(fill(final_position , num_joints), time_end);

In [ ]:
function objetivo(data::Vector{Float64})
    kp_local = data[1:num_joints]
    kv_local = data[(num_joints + 1):(num_joints * 2)]
    lambda_local = [1.,1.]
    
    ex, ev, ea, ej, tout, ta, tj = erroPDDigitalFractional(kp_local, 
        kv_local, lambda_local, xr, vr, ar, jr, robot,
        initial_state, Δt, time_end, maxTorque = saturation_value);
    return sum(iae(ej)) + sum(iae(ex))
end;

In [ ]:
ψ  = 0.7  # =>constriction factor
ϕp = 0.5  # => particle acceleration coefficient
ϕg = 0.5  # => group acceleration coefficient
iter = 150  # => iterations
num_particles = 50  # => amount of particles
min_step = 1e-4 # => stopping criterion
min_func = 1e-4 # => stopping criterion
log = false     # => log
 
particles_pso = Particles(num_particles, lb, ub);

In [ ]:
pso(particles_pso, objetivo, verbose = log, minfunc = min_func,
    minstep = min_step, psi = ψ, phip = ϕp , phig = ϕg,
    maxiter = iter, localsearch = false, neighborhood = 10)

In [ ]:
println("kp = $(particles_pso.best_position[1:num_joints])")
println("kv = $(particles_pso.best_position[(num_joints+1):(num_joints * 2)])")
println("Fitness = $(particles_pso.best_value)")

In [ ]:
plot(particles_pso.history, xlabel = "Iteration", ylabel="Fitness")